In [88]:
import dependency as dp
import importlib
importlib.reload(dp)

# project = dp.Project("/home/algor/git/raox", "ru.bmstu.rk9.rao")
# bundle = dp.Bund
# le("/home/algor/.p2/", project.module_root, project)
# bundle.update_dependencies()
bundle = dp.Bundle("/Users/algor/.p2/", "org.eclipse.swt")
bundle.update_jars()
bundle.update_dependencies()
bundle.dependencies

[b'requires.1.namespace=java.package\n', b'requires.1.name=org.eclipse.swt.accessibility2\n', b'requires.1.optional=true\n', b'requires.1.greedy=false\n', b'requires.1.range=0.0.0\n', b'\n', b'# ensure that the applicable implementation fragment gets installed (bug 361901)\n', b'requires.2.namespace = org.eclipse.equinox.p2.iu\n', b'requires.2.name = org.eclipse.swt.win32.win32.x86_64\n', b'requires.2.range = [$version$,$version$]\n', b'requires.2.filter = (&(osgi.os=win32)(osgi.ws=win32)(osgi.arch=x86_64)(!(org.eclipse.swt.buildtime=true)))\n', b'\n', b'requires.3.namespace = org.eclipse.equinox.p2.iu\n', b'requires.3.name = org.eclipse.swt.cocoa.macosx.x86_64\n', b'requires.3.range = [$version$,$version$]\n', b'requires.3.filter = (&(osgi.os=macosx)(osgi.ws=cocoa)(osgi.arch=x86_64)(!(org.eclipse.swt.buildtime=true)))\n', b'\n', b'requires.4.namespace = org.eclipse.equinox.p2.iu\n', b'requires.4.name = org.eclipse.swt.gtk.linux.x86_64\n', b'requires.4.range = [$version$,$version$]\n',

[org.eclipse.swt.accessibility2,
 org.eclipse.swt.win32.win32.x86_64,
 org.eclipse.swt.cocoa.macosx.x86_64,
 org.eclipse.swt.gtk.linux.x86_64,
 org.eclipse.swt.gtk.linux.ppc64le,
 org.eclipse.swt.gtk.linux.aarch64]

In [87]:
dp.parse_p2_info_file_lines(dp.test_dep_p2.split("\n"))

['requires.1.name=org.eclipse.swt.accessibility2', 'requires.2.name = org.eclipse.swt.win32.win32.x86_64', 'requires.3.name = org.eclipse.swt.cocoa.macosx.x86_64', 'requires.4.name = org.eclipse.swt.gtk.linux.x86_64', 'requires.5.name = org.eclipse.swt.gtk.linux.ppc64le', 'requires.6.name = org.eclipse.swt.gtk.linux.aarch64']
org.eclipse.swt.accessibility2
 org.eclipse.swt.win32.win32.x86_64
 org.eclipse.swt.cocoa.macosx.x86_64
 org.eclipse.swt.gtk.linux.x86_64
 org.eclipse.swt.gtk.linux.ppc64le
 org.eclipse.swt.gtk.linux.aarch64
['org.eclipse.swt.accessibility2', 'org.eclipse.swt.win32.win32.x86_64', 'org.eclipse.swt.cocoa.macosx.x86_64', 'org.eclipse.swt.gtk.linux.x86_64', 'org.eclipse.swt.gtk.linux.ppc64le', 'org.eclipse.swt.gtk.linux.aarch64']
[org.eclipse.swt.accessibility2, org.eclipse.swt.win32.win32.x86_64, org.eclipse.swt.cocoa.macosx.x86_64, org.eclipse.swt.gtk.linux.x86_64, org.eclipse.swt.gtk.linux.ppc64le, org.eclipse.swt.gtk.linux.aarch64]


[org.eclipse.swt.accessibility2,
 org.eclipse.swt.win32.win32.x86_64,
 org.eclipse.swt.cocoa.macosx.x86_64,
 org.eclipse.swt.gtk.linux.x86_64,
 org.eclipse.swt.gtk.linux.ppc64le,
 org.eclipse.swt.gtk.linux.aarch64]

In [74]:
import re
requires_pattern = re.compile(r"[\s]*(requires)\.[0-9]+\.(name)[\s]*\=[\s]*.*")

In [75]:
re.fullmatch(requires_pattern, "requires.3.name = org.eclipse.swt.cocoa.macosx.x86_64")

<re.Match object; span=(0, 53), match='requires.3.name = org.eclipse.swt.cocoa.macosx.x8>